# Mount google drive at this notebook

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install --upgrade gspread
!pip install pretty_html_table

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1
  Created wheel for pretty-html-table: filename=pretty_html_table-0.9.dev0-cp36-none-any.whl size=3941 sha256=d3ddcb0f53ec9b04dcb126da5a3ab8d272f959baf58df16381c8e0b1e1a97ca4
  Stored in directory: /root/.cache/pip/wheels/ca/ba/34/2df2877fbdf547891d44fd85fe8a44d8d3c6a007fb349abd9a
Successfully built pretty-html-table


# Google sheet authorization (through service account)

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

# service key json file
credentials = ServiceAccountCredentials.from_json_keyfile_name(
         'gdrive/My Drive/Scraper/scraper-300106-86075dd12529.json', scope) 

gc = gspread.authorize(credentials)
sh = gc.open("Drug Safety 0104")

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 252 kB in 2s (145 kB/s)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [ ]:
from bs4 import BeautifulSoup
from pandas import DataFrame
import datetime
import time
import pandas as pd
from gspread_dataframe import set_with_dataframe
productlist=sh.worksheet("Summary")
rows=productlist.get_all_values()
product_list=DataFrame.from_records(rows[1:],columns = rows[0])

In [ ]:
wd.get("http://readopac2.ncl.edu.tw/nclJournal/search/search.jsp?search_type=adv&la=ch")
search1 = wd.find_element_by_id("textfield1")


NoSuchElementException: ignored

In [ ]:
starttime=time.time()
uniquelen=[]
#pi=10
for pi in range(len(product_list)):
  name=product_list.iloc[pi]['Generic name']
  # go to the search webpage
  wd.get("http://readopac2.ncl.edu.tw/nclJournal/search/search.jsp?search_type=adv&la=ch")
  r=wd.page_source

  # interact with the page and fill in values
  search1 = wd.find_element_by_id("textfield1")
  search2 = wd.find_element_by_id("textfield2")
  search3 = wd.find_element_by_id("textfield3")
  dropdowns = wd.find_elements_by_name("search_op")
  option1 = dropdowns[0]
  option2 = dropdowns[1]
  check = wd.find_element_by_xpath(".//*[@value='一般性']")
  num_search = wd.find_element_by_xpath(".//*[@value='50']")
  #print(num_search.is_selected())
  search1.send_keys(product_list.iloc[pi]['Generic name'])
  search2.send_keys(product_list.iloc[pi]['English brand name'])
  search3.send_keys(product_list.iloc[pi]['Chinese brand name'])
  option1.send_keys("OR")
  option2.send_keys("OR")
  check.click()
  num_search.click()
  #print(num_search.is_selected())
  submit=wd.find_element_by_name("submit3")
  submit.click()



  # s is the result page
  s=wd.page_source

  # scrape the contents of the result page
  soup=BeautifulSoup(s)

  # check if the page contains any search result (if not go the next product in the list)
  pageresult=soup.find('td', attrs={"class","center"})
  if '查無資料' in pageresult.text:
    df = DataFrame(columns=['id','article','author','journal','date','page','retrieve date','retrieve week #','abstract link','new article or not'])
    sheet=sh.add_worksheet(name,100,26)
    set_with_dataframe(sheet,df)
    print(name+": "+str(len(df.index)))
    continue

  article = soup.select('label > a')
  if article==[]:
    totalnum=1
  else:
    a=soup.select('div > em')
    totalnum=int(a[0].text)
  itera=totalnum//50
  itera=itera+1
  towrite=DataFrame(columns=['id','article','author','journal','date','page','retrieve date','retrieve week #','abstract link','new article or not'])
  dflist=[]

  for it in range(itera):
    article = soup.select('label > a')
    article_names = []
    id=[]
    if article==[]:
      art = soup.find("div",attrs={"class","caption"})
      art=art.text
      art=art.strip()
      article_names.append(art)

      urlid=wd.current_url
      start = urlid.find("sysId=")+len("sysId=")
      end = urlid.find("&dtdId=")
      urlid=urlid[start:end]
      urlid=urlid.strip('0')
      urlid=int(urlid)
      id.append(urlid)

    else:
      for ar in article:
        article_names.append(ar.text)
      
      for x in soup.select('label > a'):
        idd=x['onclick']
        st = idd.find("goDetail('")+len("goDetail('")
        en = idd.find("','0000")
        newid=idd[st:en].strip('0')
        newid=int(newid)
        id.append(newid)


    publish_info=[]
    i=0
    headlink="http://readopac3.ncl.edu.tw"

    for details in soup.find_all('ul', attrs={"class","publishInfo"}):
      publish_info.append([])
      date_object = datetime.date.today()
      publish_info[i].append(date_object)
      weeknum=date_object.isocalendar()[1]
      publish_info[i].append(weeknum)

      for row in details.find_all('li'):
        if (row.text=='摘要'):
          link=row.find('a')
          linktext=link['href']
          complete=headlink+linktext
          publish_info[i].append(complete)
        elif (row.text):
          line=row.text
          if ('民' in line or '頁' in line or (not ':' in line and not line.isdecimal() and not '(' in line and not '下' in line)):
            publish_info[i].append(line.strip())
      if (len(publish_info[i])<7):
        publish_info[i].append(None)
      uniquelen.append(len(publish_info[i]))
      i=i+1

    # write to dataframe
    df = DataFrame(publish_info,columns=['retrieve date','retrieve week #','author','journal','date','page','abstract link'])
    df['article']=article_names
    df['id']=id
    df=df[['id','article','author','journal','date','page','retrieve date','retrieve week #','abstract link',]]
    df['new article or not']=''

    dflist.append(df)

    # go to the next page (if exists)
    if it!=itera-1:
      pn=it+2
      pagenum=str(pn)
      xpath="//*[text()='"+pagenum+"']"
      secondpage=wd.find_element_by_xpath(xpath)
      secondpage.click()
      k=wd.page_source
      soup=BeautifulSoup(k)

  towrite=pd.concat(dflist)
  # Open our new sheet and add some data.
  sheet=sh.add_worksheet(name,300,26)

  # fill the worksheet with entries in dataframe df
  set_with_dataframe(sheet,towrite)
  print(name+": "+str(len(towrite.index)))
sh.del_worksheet(worksheet)
endtime=time.time()
print(endtime-starttime)

NoSuchElementException: ignored

In [ ]:
# get all sheet names in one spread sheet
sheets=sh.worksheets()
sheet_names=str(sheets)

In [ ]:
emaillist=gc.open("Drug safety email list").sheet1
all=emaillist.get_all_values()
rec=""
first=True
for a in all:
  if first:
    rec=rec+a[0]
    first=False
  else:
    rec=rec+","+a[0]

In [ ]:
summary=sh.worksheet("Summary")
lists=summary.get_all_values()
summarydf=DataFrame.from_records(lists[1:],columns = lists[0])

In [ ]:
oldcount=[]
oldcount.append(1)
oldcount

In [ ]:
soup

In [ ]:
summarydf.iloc[pi]['English brand name']

In [ ]:
BeautifulSoup(s)

In [ ]:
'''
starttime=time.time()
uniquelen=[]
flag=0
emaildf=DataFrame(columns=['product name','id','article','author','journal','date','page','retrieve date','retrive week #','abstract link','new article or not'])
summary=sh.worksheet("Summary")
lists=summary.get_all_values()
summarydf=DataFrame.from_records(lists[1:],columns = lists[0])
oldcount=[]
newcount=[]
numnew=0
pi=21
#for pi in range(len(summarydf)):
name=summarydf.iloc[pi]['Generic name']
#if summarydf.iloc[pi]['Active'] !="Yes":
  #continue

if name not in sheet_names:
  sheet=sh.add_worksheet(name, 300, 26)
  newwrite=DataFrame(columns=['id','article','author','journal','date','page','retrieve date','retrive week #','abstract link','new article or not'])
  set_with_dataframe(sheet,newwrite)
sheet = sh.worksheet(name)
rows=sheet.get_all_values()
olddf=DataFrame.from_records(rows[1:],columns = rows[0])
olddf['new article or not']=''

oldcount.append(len(olddf.index))

values_list = sheet.col_values(1)
for i in range(1,len(values_list)):
  values_list[i]=int(values_list[i])
#id_values=pd.DataFrame(values_list[1:len(values_list)],columns=[values_list[0]])

#name="Methoxy polyethylene glycol-epoetin beta"
# go to the search webpage
wd.get("http://readopac2.ncl.edu.tw/nclJournal/search/search.jsp?search_type=adv&la=ch")
r=wd.page_source

# interact with the page and fill in values
search1 = wd.find_element_by_id("textfield1")
search2 = wd.find_element_by_id("textfield2")
search3 = wd.find_element_by_id("textfield3")
dropdowns = wd.find_elements_by_name("search_op")
option1 = dropdowns[0]
option2 = dropdowns[1]
check = wd.find_element_by_xpath(".//*[@value='一般性']")
num_search = wd.find_element_by_xpath(".//*[@value='50']")

search1.send_keys(name)
search1.send_keys(summarydf.iloc[pi]['Generic name'])
search2.send_keys(summarydf.iloc[pi]['English brand name'])
search3.send_keys(summarydf.iloc[pi]['Chinese brand name'])
option1.send_keys("OR")
option2.send_keys("OR")
check.click()
num_search.click()

submit=wd.find_element_by_name("submit3")
submit.click()

# s is the result page
s=wd.page_source

# scrape the contents of the result page
soup=BeautifulSoup(s)

# check if the page contains any search result (if not go the next product in the list)
pageresult=soup.find('td', attrs={"class","center"})
#if '查無資料' in pageresult.text:
  #newcount.append(0)
  #continue

article = soup.select('label > a')
if article==[]:
  totalnum=1
else:
  a=soup.select('div > em')
  totalnum=int(a[0].text)
itera=totalnum//50
itera=itera+1
towrite=DataFrame(columns=['id','article','author','journal','date','page','retrieve date','retrive week #','abstract link','new article or not'])
dflist=[]

for it in range(itera):
  article = soup.select('label > a')
  article_names = []
  id=[]
  if article==[]:
    art = soup.find("div",attrs={"class","caption"})
    art=art.text
    art=art.strip()
    article_names.append(art)

    urlid=wd.current_url
    start = urlid.find("sysId=")+len("sysId=")
    end = urlid.find("&dtdId=")
    urlid=urlid[start:end]
    urlid=urlid.strip('0')
    urlid=int(urlid)
    id.append(urlid)

  else:
    for ar in article:
      article_names.append(ar.text)
    
    for x in soup.select('label > a'):
      idd=x['onclick']
      st = idd.find("goDetail('")+len("goDetail('")
      en = idd.find("','0000")
      newid=idd[st:en].strip('0')
      newid=int(newid)
      id.append(newid)


  publish_info=[]
  i=0
  headlink="http://readopac3.ncl.edu.tw"
  for details in soup.find_all('ul', attrs={"class","publishInfo"}):
    publish_info.append([])
    date_object = datetime.date.today()
    publish_info[i].append(date_object)
    weeknum=date_object.isocalendar()[1]
    publish_info[i].append(weeknum)

    for row in details.find_all('li'):
      if (row.text=='摘要'):
        link=row.find('a')
        linktext=link['href']
        complete=headlink+linktext
        publish_info[i].append(complete)
      elif (row.text):
        line=row.text
        if ('民' in line or '頁' in line or (not ':' in line and not line.isdecimal() and not '(' in line and not '下' in line)):
          publish_info[i].append(line.strip())
    if (len(publish_info[i])<7):
      publish_info[i].append(None)
    uniquelen.append(len(publish_info[i]))
    i=i+1

  # write to dataframe
  df = DataFrame(publish_info,columns=['retrieve date','retrieve week #','author','journal','date','page','abstract link'])
  df['article']=article_names
  df['id']=id
  df=df[['id','article','author','journal','date','page','retrieve date','retrieve week #','abstract link']]

  dflist.append(df)

  # go to the next page (if exists)
  if it!=itera-1:
    pn=it+2
    pagenum=str(pn)
    xpath="//*[text()='"+pagenum+"']"
    secondpage=wd.find_element_by_xpath(xpath)
    secondpage.click()
    k=wd.page_source
    soup=BeautifulSoup(k)  

towrite=pd.concat(dflist)

idlist=towrite['id']
towrite=towrite.set_index("id",inplace=False,drop=False)  
olddf=olddf.set_index("id",inplace=False,drop=False)
emaildf=emaildf.set_index("id",inplace=False,drop=False)

newentry=DataFrame(columns=['id','article','author','journal','date','page','retrieve date','retrieve week #','abstract link','new article or not'])
for a in idlist:
  if a not in values_list:
    flag=1
    towrite.loc[a,'new article or not']='new'
    newf=towrite.loc[a]
    newentry=newentry.append(newf)
    #newf=newf[['product name','id','article','author','journal','date','page','retrieve date','retrive week #','abstract link','new article or not']]
    emaildf=emaildf.append(newf)
    emaildf.loc[a,'product name']=name
    numnew=numnew+1
newentry=newentry.append(olddf)

#  print(name+": "+str(len(newentry.index)))
newcount.append(len(newentry.index))
newentry=newentry[['id','article','author','journal','date','page','retrieve date','retrieve week #','abstract link','new article or not']]

# fill the worksheet with entries in dataframe df
set_with_dataframe(sheet,newentry)


#sh.del_worksheet(worksheet)
summarydf['# of search results last week']=oldcount
summarydf['# of search results this week']=newcount
set_with_dataframe(summary,summarydf)
endtime=time.time()
print(endtime-starttime)
'''

In [ ]:
sheet = sh.worksheet('Cobimetinib')
rows=sheet.get_all_values()
olddf=DataFrame.from_records(rows[1:],columns = rows[0])
olddf

In [ ]:
set_with_dataframe(summary,summarydf)

In [ ]:
newentry=DataFrame(columns=['id','article','author','journal','date','page','retrieve date','retrieve week #','abstract link','new article or not'])
newentry=newentry.append(olddf)
len(newentry.index)

In [ ]:
from smtplib import SMTP
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from pretty_html_table import build_table

In [ ]:
def send_mail(body):
  message = MIMEMultipart()
  year = datetime.date.today().isocalendar()[0]
  message['Subject'] = str(year)+' Week '+str(weeknum)+' update'
  message['From'] = 'ann.chao@roche.com'
  recipients = rec
  message['To'] = recipients

  bodt_text="Number of new articles this week: "+str(numnew)
  url="<p>Click <a href='https://docs.google.com/spreadsheets/d/1hFoe-tSbN1r4qNIXFODXfFllkQpJlV28_ms9ZfqhOII/edit?usp=sharing'>here</a> to view the complete google sheet</p>"
  body_content = body
  message.attach(MIMEText(bodt_text))
  message.attach(MIMEText(url,"html"))
  message.attach(MIMEText(body_content, "html"))
  msg_body = message.as_string()

  server = SMTP('smtp.gmail.com', 587)
  server.starttls()
  server.login(message['From'], 'rongNan325')
  server.sendmail(message['From'], recipients.split(','), msg_body)
  server.quit()

In [ ]:
def send_mail_noupdate():
  message = MIMEMultipart()
  year = datetime.date.today().isocalendar()[0]
  message['Subject'] = str(year)+' Week '+str(weeknum)+' update'
  message['From'] = 'ann.chao@roche.com'
  recipients = rec
  message['To'] = recipients

  bodt_text="Number of new articles this week: "+str(numnew)
  url="<p>Click <a href='https://docs.google.com/spreadsheets/d/1hFoe-tSbN1r4qNIXFODXfFllkQpJlV28_ms9ZfqhOII/edit?usp=sharing'>here</a> to view the complete google sheet</p>"
  message.attach(MIMEText(bodt_text))
  message.attach(MIMEText(url,"html"))
  msg_body = message.as_string()

  server = SMTP('smtp.gmail.com', 587)
  server.starttls()
  server.login(message['From'], 'rongNan325')
  server.sendmail(message['From'], recipients.split(','), msg_body)
  server.quit()

In [ ]:
flag=0
weeknum=53
rec='ann.chao@roche.com,miojasonann@gmail.com'
numnew=0

In [ ]:
if flag==1:
  emaildf=emaildf[['product name','article','author','journal','date','page','abstract link']]
  mail_message_body = build_table(emaildf,'blue_light')
  send_mail(mail_message_body)
else:
  send_mail_noupdate()

In [ ]:
"""
if flag ==1:
  # Import Python Packages
  import smtplib
  # Set Global Variables
  gmail_user = 'ann.chao@roche.com'
  gmail_password = ''
  # Create Email 
  mail_from = gmail_user
  mail_to = ['ann.chao@roche.com','miojasonann@gmail.com']
  mail_subject = 'Week '+ weeknum
  mail_message_body = 'Hello World!'

  mail_message = '''\
  From: %s
  To: %s
  Subject: %s
  %s
  ''' % (mail_from, mail_to, mail_subject, mail_message_body)
  # Sent Email
  server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
  server.login(gmail_user, gmail_password)
  server.sendmail(mail_from, mail_to, mail_message)
  server.close()
"""

In [ ]:
import datetime
datetime.date.today().isocalendar()[1]